In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

from InteriorHelmholtzSolver2D import *
from InteriorHelmholtzSolverRAD import *

c         = 344.0 # speed of sound [m/s]
rho       = 1.205 # density of air [kg/m^3]
frequency = 40.0 # frequency [Hz]

print "c   = ", c
print "rho = ", rho
print "f   = ", frequency

In [ ]:
aVertex = np.array([[0.000, 1.000], [0.174, 0.985], [0.342, 0.940], [0.500, 0.866],
                    [0.643, 0.766], [0.766, 0.643], [0.866, 0.500], [0.940, 0.342],
                    [0.985, 0.174], [1.000, 0.000], [0.985,-0.174], [0.940,-0.342],
                    [0.866,-0.500], [0.766,-0.643], [0.643,-0.766], [0.500,-0.866],
                    [0.342,-0.940], [0.174,-0.985], [0.000,-1.000]], dtype=np.float32)

aEdge = np.array([[0, 1],   [1, 2],   [2, 3],   [3, 4],
                  [4, 5],   [5, 6],   [6, 7],   [7, 8],
                  [8, 9],   [9, 10],  [10, 11], [11, 12],
                  [12, 13], [13, 14], [14, 15], [15, 16],
                  [16, 17], [17, 18]], dtype=np.int32)

interiorPoints = np.array([[0.0000,  0.0000],
                           [0.0000,  0.5000],
                           [0.0000, -0.5000],
                           [0.5000,  0.0000]], dtype=np.float32)


In [ ]:
centers = 0.5 * (aVertex[aEdge[:, 0]] + aVertex[aEdge[:, 1]])

In [ ]:
# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#

solver = InteriorHelmholtzSolverRAD(aVertex, aEdge)

boundaryCondition = BoundaryCondition(aEdge.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)
boundaryCondition.f

k = solver.frequencyToWavenumber(frequency)

boundaryCondition.f[:] = np.sin(k * centers[:,1])

boundaryIncidence = BoundaryIncidence(aEdge.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
for i in range(centers.shape[0]):
    z = centers[i, 1]
    n = solver.Normal2D(aVertex[aEdge[i,0],:], aVertex[aEdge[i,1],:])
    boundaryCondition.f[i] = k * np.cos(k*z) * n[1]
    
boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)

# Test Problem 3
# Dirichlet boundary condition, such that phi = sin(k/ sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

zp = 0.25
for i in range(centers.shape[0]):
    r = centers[i, 0]
    z = centers[i, 1]
    # make input complex so proper sqrt is called
    rpq = np.sqrt(0.0j + r**2 + (z - zp)**2)
    boundaryCondition.f[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
    boundaryIncidence.phi[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
    n = solver.Normal2D(aVertex[aEdge[i,0],:], aVertex[aEdge[i,1],:])
    drbdn = (r * n[0] + (z - zp) * n[1]) / rpq
    boundaryIncidence.v[i] = drbdn * np.exp(1j * k * rpq) * (1j * k * rpq - 1.0) \
        / (4.0 * np.pi * rpq*rpq)
    
for i in range(interiorPoints.shape[0]):
    r = interiorPoints[i, 0]
    z = interiorPoints[i, 1]
    # make input complex so proper sqrt is called
    rpq = np.sqrt(0.0j + r**2 + (zp - z)**2)
    interiorIncidentPhi[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
       
print "Starting boundary solution."
boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
print "Completed boundary solution."
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "Completed interior solution."
print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)
 